In [7]:
import cv2
import numpy as np
import csv
import os

# video_path='training01.mp4'


def track_and_save_hexbug(video_path, output_csv='hexbug_path18.csv', output_video='hexbug_tracked18.avi',
                          min_motion_threshold=2.0):
    cap = cv2.VideoCapture('training018.mp4')
    ret, frame = cap.read()
    if not ret:
        print("❌ 无法读取视频")
        return

    prev_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    h, w = prev_gray.shape

    # 初始化视频写入器
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out_vid = cv2.VideoWriter(output_video, fourcc, cap.get(cv2.CAP_PROP_FPS), (w, h))

    # 轨迹坐标
    trajectory = []

    # 初始化 CSV 写入器
    with open(output_csv, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['', 't', 'hexbug', 'x', 'y'])

      # trajectory = [] #丢第一帧新加
        index = 0
        t = 0
        
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        while index < frame_count:
          
          #if frame is None:
                #break
          ret, frame = cap.read()
          if not ret:
            print(f"⚠️ 第 {index} 帧读取失败")
            writer.writerow([index, t, 0, -1, -1])
            index += 1
            t += 1
            continue
            #break

          gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
          gray = cv2.equalizeHist(gray)  # 增强光流输入图像质量
          hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
          lower_green = np.array([40, 40, 40])      # Hue 40-80 之间通常为绿色
          upper_green = np.array([80, 255, 255])
          color_mask = cv2.inRange(hsv, lower_green, upper_green)

         #  hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)      #以下四行加入紫色锁定
         #  lower_purple = np.array([125, 40, 40]) #50 change 40
         #  upper_purple = np.array([155, 255, 255])
         #  color_mask = cv2.inRange(hsv, lower_purple, upper_purple)

 # ⚠️ 第二帧以后才有 prev_gray，第一帧没有光流
          if index == 0: #丢帧新加

             contours, _ = cv2.findContours(color_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

             # ✅ 面积过滤
             contours = [cnt for cnt in contours if cv2.contourArea(cnt) > 100] #后加

             if contours:
                    largest = max(contours, key=cv2.contourArea)
                    M = cv2.moments(largest)
                    if M["m00"] != 0:
                        cx = int(M["m10"] / M["m00"])
                        cy = int(M["m01"] / M["m00"])
                        trajectory.append((cx, cy))
                        cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)
                        writer.writerow([index, t, 0, cx, cy])
                    else:
                        writer.writerow([index, t, 0, None, None])
             else:
                    writer.writerow([index, t, 0, None, None])

          else:
            # 计算光流
            flow = cv2.calcOpticalFlowFarneback(prev_gray, gray,
                                                None, 0.5, 3, 15, 3, 5, 1.2, 0)
            mag,_ = cv2.cartToPolar(flow[..., 0], flow[..., 1])
       #修改     mask = mag > min_motion_threshold

       #修改     motion_mask = np.uint8(mask) * 255
            _, motion_mask = cv2.threshold(mag, min_motion_threshold, 255, cv2.THRESH_BINARY)
            motion_mask = motion_mask.astype(np.uint8)  # 必须是二值图像

    #修改替换   contours, _ = cv2.findContours(motion_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            combined_mask = cv2.bitwise_and(motion_mask, color_mask)  # 上一行替换以下两行
            contours, _ = cv2.findContours(combined_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)


            if contours:
                # 最大移动区域视为 hexbug
                largest = max(contours, key=cv2.contourArea)
                M = cv2.moments(largest)
                if M["m00"] != 0:
                    cx = int(M["m10"] / M["m00"])
                    cy = int(M["m01"] / M["m00"])

                    # 画中心点
                    trajectory.append((cx, cy))
                    for i in range(1, len(trajectory)):
                        cv2.line(frame, trajectory[i-1], trajectory[i], (0, 255, 0), 2)
                    cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)

                    writer.writerow([index, t, 0, cx, cy])
                else:
                    writer.writerow([index, t, 0, None, None]) #none 改为-1
            else:

                writer.writerow([index, t, 0, None, None])   #none 改为-1


          out_vid.write(frame)

          prev_gray = gray
          #ret, frame = cap.read()
          #if not ret:
            # break
          index += 1
          t += 1


          #  index += 1
          #  t += 1
          #  out_vid.write(frame)
          #  prev_gray = gray

        print(f"✅ 自动追踪完成，轨迹视频已保存为：{output_video}")
        print(f"✅ 结果已保存为 CSV 文件：{output_csv}")
        print(f"处理帧数: {index}, 视频总帧数: {int(cap.get(cv2.CAP_PROP_FRAME_COUNT))}") #修改打印帧数
        

    cap.release()
    out_vid.release()
    #cv2.destroyAllWindows()#后加
# 示例调用
track_and_save_hexbug('training018.mp4')

⚠️ 第 100 帧读取失败
✅ 自动追踪完成，轨迹视频已保存为：hexbug_tracked18.avi
✅ 结果已保存为 CSV 文件：hexbug_path18.csv
处理帧数: 101, 视频总帧数: 101


In [8]:
import cv2
import numpy as np
import csv

def track_and_save_hexbug_head(video_path, output_csv='hexbug_head_path.csv', output_video='hexbug_head_tracked.avi',
                              min_motion_threshold=2.0, head_region_height_ratio=1.0):
    """
    head_region_height_ratio: 定义头部区域占轮廓高度的比例，通常取0.3表示轮廓顶部30%区域为头部
    """
    cap = cv2.VideoCapture(video_path)
    ret, frame = cap.read()
    if not ret:
        print("❌ 无法读取视频")
        return

    prev_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    h, w = prev_gray.shape

    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out_vid = cv2.VideoWriter(output_video, fourcc, cap.get(cv2.CAP_PROP_FPS), (w, h))

    trajectory = []

    with open(output_csv, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['', 't', 'hexbug', 'x', 'y'])

        index = 0
        t = 0
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        while index < frame_count:
            ret, frame = cap.read()
            if not ret:
                print(f"⚠️ 第 {index} 帧读取失败")
                writer.writerow([index, t, 0, -1, -1])
                index += 1
                t += 1
                continue

            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            gray = cv2.equalizeHist(gray)  # 提升图像对比度

            hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
            lower_green = np.array([40, 40, 40])
            upper_green = np.array([80, 255, 255])
            color_mask = cv2.inRange(hsv, lower_green, upper_green)

            if index == 0:
                # 第一帧直接找最大轮廓中心
                contours, _ = cv2.findContours(color_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                contours = [c for c in contours if cv2.contourArea(c) > 100]

                if contours:
                    largest = max(contours, key=cv2.contourArea)
                    M = cv2.moments(largest)
                    if M["m00"] != 0:
                        cx = int(M["m10"] / M["m00"])
                        cy = int(M["m01"] / M["m00"])
                        trajectory.append((cx, cy))
                        cv2.circle(frame, (cx, cy), 5, (0, 0, 255), -1)
                        writer.writerow([index, t, 0, cx, cy])
                    else:
                        writer.writerow([index, t, 0, -1, -1])
                else:
                    writer.writerow([index, t, 0, -1, -1])
            else:
                flow = cv2.calcOpticalFlowFarneback(prev_gray, gray,
                                                    None, 0.5, 3, 15, 3, 5, 1.2, 0)
                mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])

                _, motion_mask = cv2.threshold(mag, min_motion_threshold, 255, cv2.THRESH_BINARY)
                motion_mask = motion_mask.astype(np.uint8)

                combined_mask = cv2.bitwise_and(motion_mask, color_mask)

                contours, _ = cv2.findContours(combined_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

                if contours:
                    largest = max(contours, key=cv2.contourArea)
                    x, y, w_contour, h_contour = cv2.boundingRect(largest)

                    # 计算头部区域 y 坐标范围
                    head_y_end = y + int(h_contour * head_region_height_ratio)

                    # 构造头部掩码（只关注轮廓顶部30%）
                    head_mask = np.zeros_like(combined_mask)
                    # 在头部区域内填充轮廓部分
                    contour_mask = np.zeros_like(combined_mask)
                    cv2.drawContours(contour_mask, [largest], -1, 255, thickness=cv2.FILLED)
                    head_mask[y:head_y_end, x:x+w_contour] = contour_mask[y:head_y_end, x:x+w_contour]

                    # 头部内运动掩码 = 运动掩码 & 头部掩码
                    head_motion_mask = cv2.bitwise_and(motion_mask, head_mask)

                    # 在头部运动掩码中找最大运动点
                    ys, xs = np.where(head_motion_mask > 0)
                    if len(xs) > 0:
                        # 找运动幅度最大的点坐标
                        max_idx = np.argmax(mag[ys, xs])
                        cx = xs[max_idx]
                        cy = ys[max_idx]
                    else:
                        # 如果头部没检测到运动点，退回到轮廓中心
                        M = cv2.moments(largest)
                        if M["m00"] != 0:
                            cx = int(M["m10"] / M["m00"])
                            cy = int(M["m01"] / M["m00"])
                        else:
                            cx, cy = -1, -1

                    trajectory.append((cx, cy))

                    # 绘制轨迹
                    for i in range(1, len(trajectory)):
                        cv2.line(frame, trajectory[i-1], trajectory[i], (0, 255, 0), 2)
                    if cx >= 0 and cy >= 0:
                        cv2.circle(frame, (cx, cy), 5, (0, 0, 255), -1)

                    writer.writerow([index, t, 0, cx, cy])
                else:
                    writer.writerow([index, t, 0, -1, -1])

            out_vid.write(frame)
            prev_gray = gray
            index += 1
            t += 1

        print(f"✅ 跟踪完成，轨迹视频保存为：{output_video}")
        print(f"✅ 结果保存为 CSV 文件：{output_csv}")
        print(f"处理帧数: {index}, 视频总帧数: {frame_count}")

    cap.release()
    out_vid.release()


# 示例调用，请替换为你的视频路径
track_and_save_hexbug_head('training018.mp4')


⚠️ 第 100 帧读取失败
✅ 跟踪完成，轨迹视频保存为：hexbug_head_tracked.avi
✅ 结果保存为 CSV 文件：hexbug_head_path.csv
处理帧数: 101, 视频总帧数: 101
